 Projekt: GenAI/Conversational AI Formular
Ausfüllassistent PoC (Proof of Concept)
 Beschreibung
 Oft gibt es Formulare oder Fragebögen, die sehr spezifische Fragen
 beinhalten und mit denen sich User:innen überfordert fühlen
 Gerne hätten sie jemanden, um näher Fragen stellen zu können oder
 Vorschläge/Beispiele geben zu können
 Ein GenAI-Assistent hat das Potenzial, hier eine automatisierte beliebig
 vervielfältigbare Lösung darzustellen
 Conversational AI ist einer der Use Cases mit meistem Potenzial
 PoC = Minimal; zeigen, dass etwas geht oder nicht geht oder vielleicht gehen
 könnte
 Umsetzung
 alles in ein Jupyter Notebook und während der Präsentation einzelne Schritte
 zeigen, wie diese bei so einem potenziellen Ausfüllassistenten aussehen
 könnte
 so wie wir es im Unterricht gemacht haben, nur spezieller bezogen auf euren
 UseCase
 keine fertige Anwendung, eher erforschen, wie etwas möglich sein könnte,
 warum, und warum nicht
 da wir mit kleinen Models lokal arbeiten, begrenzt in der Ausreizung der
 Modelle - daher lieber einzelne Schritte anstatt komplette Anwendung
 könnte man bei der Befüllung eines Feldes ein RAG-Tool nutzen?
 könnte ein Nutzer sich Beispiele geben lassen mit Hilfe eines RAG-Tools?

 UC12. Lebensmittel-Beratungsassistent
 Felder
 Alter (int)
 Geschlecht (str, z.B. männlich, weiblich, divers)
 Körpergewicht in kg (float)
 Körpergröße in cm (float)
 Allergien oder Unverträglichkeiten (Optional[str])
 Essgewohnheiten (str, z.B. vegetarisch, vegan, omnivor, paleo)
 Ziel (str, z.B. abnehmen, Muskelaufbau, Gewicht halten, fitter werden)
 Körperliche Aktivität (str, z.B. wenig, moderat, sportlich)
 Zeitaufwand für Kochen pro Tag in Minuten (int)
 Gesundheitszustand / Vorerkrankungen (Optional[str])
 Documents
 20 Seiten mit Infos zu Diätformen, Ernährungsempfehlungen,
 Kontraindikationen (z. B. Low Carb bei Nierenproblemen), Beispiel
Tagespläne für verschiedene Ernährungstypen.
 Mögliche Tools
 RAG für Abruf passender Diäten und Beispielpläne
BMI-Rechner
 Kalorienbedarfsrechner
 Allergie-Check-Tool
 Diät-Vergleichstool nach Ziel und Einschränkungen
 Mögliche User:innen-Nachfrage-Prompts
 „Was passt besser zu mir: Low Carb oder Mittelmeerdiät?“ -> RAG +
 Diät-Vergleich
 „Ich bin 1,75 m groß und wiege 95 kg – wie hoch ist mein BMI?“ -> BMI
Rechner
 „Wie viele Kalorien brauche ich bei wenig Bewegung?“ ->
 Kalorienbedarfsrechner
 „Ich bin laktoseintolerant – welche Diäten schließen Milchprodukte aus?“-> RAG + Allergie-Check
 „Kannst du mein Ziel ‚fitter werden‘ klarer formulieren?“ -> Kein Tool
Aufruf
 1B) Im Forums-Thread Auswahl mitteilen - First Come
 First Serve
 https://moodle.hochschule-burgenland.at/mod/forum/discuss.php?d=77135
 2. RAG-Komponente aufbauen
 2A) Dokumente (ca. 20 Seiten) zusammenstellen
 Was z.B.?
 Webseiten to PDFs
 selbst kreierte Texte
 von KI-Chat-Anwendung synthetisch erzeugte Texte
 im Web gefundene PDFs
 Wie?
 wenn mehrere Doks, alle in ein 20-Seiten-PDF mergen
 Auf was achten?
 hier an Felder denken, was nützlich sein könnte für Befüllung oder
 Beratung oder Beispielgebung
 2B) 20-Seiten-PDF prozessieren (Parsing, Chunking)
 2C) Chunks einbetten und in kreierte Vektor-DB (Milvus
 Lite oder Andere) indexieren (Embedding, Indexing)
3. Agents und Tools ausprobieren
 Beispielcode
 # Pydantic Felder
 class SupportDevice(BaseModel):
    device_type: str = Field(description="Type of device (e.g., Printer, Lap
    model_number: str = Field(description="Model number of the device")
    os_version: str = Field(description="Operating system version")
    error_code: Optional[str] = Field(None, description="Error code if appli
    recurring_error: bool = Field(description="Whether the error occurs regu
    battery_life_hours: Optional[int] = Field(None, description="Battery lif
    last_working_date: str = Field(description="Date when the device last wo
    device_restarted: bool = Field(description="Whether the device has been 
    issue_description: str = Field(description="Detailed description of the 
    warranty_valid: bool = Field(description="Whether the warranty is still 
# FileSearchTool Beispiel mit Anbindung an Vector DB (hier https://openai.gi
 from agents import Agent, Runner, FileSearchTool
 # Instantiate the file search tool with your Milvus Lite vector store id.
 file_search_tool = FileSearchTool(
    vector_store_ids=["milvus_lite_store_id"],  # Replace with your actual v
    max_num_results=5,                          # Maximum number of results 
    include_search_results=True                 # Include search results in 
    # ranking_options and filters can be set here if needed
 )
 # Create an agent that uses the FileSearchTool.
 agent = Agent(
    name="File Search Agent",
    instructions="You are a helpful assistant that can search for documents 
                 "When a query is received, use the FileSearchTool to find t
    tools=[file_search_tool],
    model="gpt-4o"  # Ensure you use a supported OpenAI model (e.g., GPT-4o)
 )
 # Run the agent with a sample query.
 response = Runner.run(agent, "Find documents related to machine learning res
 print(response)

In [ ]:
from __future__ import annotations
from aiworkshop_utils.standardlib_imports import os, json, base64, logging, Optional, List, Literal, pprint, glob, asyncio, datetime, date, time, timezone, ZoneInfo, uuid, dataclass
from aiworkshop_utils.thirdparty_imports import AutoTokenizer, load_dotenv, requests, BaseModel, Field, pd, cosine_similarity, plt, np, DataType, MilvusClient, DDGS, rprint
from aiworkshop_utils.custom_utils import show_pretty_json, encode_image
from aiworkshop_utils.jupyter_imports import Markdown, HTML, JSON, display, widgets
from aiworkshop_utils.openai_imports import OpenAI, Agent, Runner, InputGuardrail, GuardrailFunctionOutput, InputGuardrailTripwireTriggered, OpenAIChatCompletionsModel, AsyncOpenAI, set_tracing_disabled, ModelSettings, function_tool, trace, ResponseContentPartDoneEvent, ResponseTextDeltaEvent, RawResponsesStreamEvent, TResponseInputItem, ItemHelpers, MessageOutputItem, RunContextWrapper, input_guardrail, output_guardrail
from aiworkshop_utils import config




In [33]:


endpoint_base = config.OLLAPI_ENDPOINT_BASE
model_name = config.OMODEL_LLAMA3D2 
set_tracing_disabled(True)

generative_model = OpenAIChatCompletionsModel(
    model=model_name,
    openai_client=AsyncOpenAI(base_url=endpoint_base, api_key="fake-key")
)

In [34]:
class FoodForm(BaseModel):
    alter: int = Field(..., description="Alter in Jahren")
    geschlecht: str = Field(..., description="Geschlecht: männlich, weiblich, divers")
    koerpergewicht: float = Field(..., description="Körpergewicht in kg")
    koerpergroesse: float = Field(..., description="Körpergröße in cm")
    allergien: Optional[str] = Field(None, description="Allergien oder Unverträglichkeiten")
    essgewohnheiten: str = Field(..., description="Essgewohnheiten: vegetarisch, vegan, omnivor, paleo")
    ziel: str = Field(..., description="Ziel: abnehmen, Muskelaufbau, Gewicht halten, fitter werden")
    koerperliche_aktivitaet: str = Field(..., description="Körperliche Aktivität: wenig, moderat, sportlich")
    kochen_zeitaufwand: int = Field(..., description="Zeitaufwand fürs tägliche Kochen in Minuten")
    gesundheit: Optional[str] = Field(None, description="Gesundheitszustand oder Vorerkrankungen")

# Beispiel: Formularinstanz erzeugen und ausgeben
sample_form = FoodForm(
    alter=30,
    geschlecht="männlich",
    koerpergewicht=85.0,
    koerpergroesse=180.0,
    allergien="keine",
    essgewohnheiten="omnivor",
    ziel="Muskelaufbau",
    koerperliche_aktivitaet="sportlich",
    kochen_zeitaufwand=45,
    gesundheit="leicht erhöhter Blutdruck"
)


In [40]:
# generative_model Setup
endpoint_base = config.OLLAPI_ENDPOINT_BASE
model_name = config.OMODEL_LLAMA3D2
generative_model = OpenAIChatCompletionsModel(
    model=model_name,
    openai_client=AsyncOpenAI(base_url=endpoint_base, api_key="fake-key")
)

# FoodForm Klasse (bereits vorhanden)
class FoodForm(BaseModel):
    alter: int = Field(..., description="Alter in Jahren")
    geschlecht: str = Field(..., description="Geschlecht: männlich, weiblich, divers")
    koerpergewicht: float = Field(..., description="Körpergewicht in kg")
    koerpergroesse: float = Field(..., description="Körpergröße in cm")
    allergien: Optional[str] = Field(None, description="Allergien oder Unverträglichkeiten")
    essgewohnheiten: str = Field(..., description="Essgewohnheiten: vegetarisch, vegan, omnivor, paleo")
    ziel: str = Field(..., description="Ziel: abnehmen, Muskelaufbau, Gewicht halten, fitter werden")
    koerperliche_aktivitaet: str = Field(..., description="Körperliche Aktivität: wenig, moderat, sportlich")
    kochen_zeitaufwand: int = Field(..., description="Zeitaufwand fürs tägliche Kochen in Minuten")
    gesundheit: Optional[str] = Field(None, description="Gesundheitszustand oder Vorerkrankungen")

# Funktion zum Extrahieren von Daten aus einem Prompt
async def extract_food_form_from_prompt(prompt: str) -> FoodForm:
    data = {
        "model": model_name,
        "prompt": prompt + "\nAntworte in JSON, passend zum Schema der FoodForm-Klasse.",
        "stream": False,
        "format": FoodForm.model_json_schema()
    }
    response = requests.post(config.OAPI_GENERATE_URL, json=data)
    response_json = response.json()
    response_data = json.loads(response_json["response"])
    return FoodForm(**response_data)

# Beispiel-Tools (bereits vorhanden)
@function_tool
def bmi_calculator(gewicht: float, groesse: float) -> dict:
    bmi = gewicht / ((groesse / 100) ** 2)
    return {"bmi": round(bmi, 2)}

# Agent Setup
food_assistant_agent = Agent(
    name="Food Assistant",
    instructions="Du bist ein Lebensmittel-Beratungsassistent. Extrahiere Daten aus dem Prompt und nutze Tools wie den BMI-Rechner.",
    model=generative_model,
    tools=[bmi_calculator]
)

# Hauptworkflow
async def main():
    prompt = ("Ich bin 30 Jahre alt, männlich, 1,80 m groß, wiege 85 kg, esse omnivor, "
              "möchte Muskelaufbau, bin sportlich aktiv, habe 45 Minuten zum Kochen und keine Allergien.")
    
    # Daten aus Prompt extrahieren
    form_data = await extract_food_form_from_prompt(prompt)
    print("----- Extrahierte Daten -----")
    print(form_data.model_dump_json(indent=4))
    
    # Beispiel: BMI berechnen
    bmi_query = f"Berechne meinen BMI. Meine Daten: {form_data.json()}"
    bmi_result = await Runner.run(food_assistant_agent, bmi_query)
    print("----- BMI-Ergebnis -----")
    print(bmi_result.final_output)

await main()

----- Extrahierte Daten -----
{
    "alter": 30,
    "geschlecht": "männlich",
    "koerpergewicht": 85.0,
    "koerpergroesse": 1.0,
    "allergien": "keine",
    "essgewohnheiten": "omnivorisch",
    "ziel": "Muskelaufbau",
    "koerperliche_aktivitaet": "sportlich aktiv",
    "kochen_zeitaufwand": 45,
    "gesundheit": null
}


C:\Users\Admin\AppData\Local\Temp\ipykernel_10936\1820052355.py:60: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  bmi_query = f"Berechne meinen BMI. Meine Daten: {form_data.json()}"


----- BMI-Ergebnis -----
Ich möchte nun Ihren BMI berechnen.

Gemäß Ihren Angaben:

* Alter: 30
* Geschlecht: männlich
* Körpergewicht: 85,0 kg
* Körpergröße: 1 m (ich wird angenommen, dass Sie etwa so groß sind wie die durchschnittliche deutsche Erwachsenenmasse)
* Koerperliche Aktivitaet: sportlich aktiv

Um Ihren BMI zu berechnen, verwende ich den folgenden Rechner:

BMI = Körpergewicht (kg) / (Körpergröße (m))^2

Beim Einsetzen Ihrer Werte ergibt sich ein BMI von: 850000
